In [3]:
from solver import processing

In [ ]:
### FEATURE PARAMETERS
# prediction target
feature_predict = 'P_avg'
# prediction inputs from turbine data
features_train = ['P_avg', 'Ba_avg', 'Wa_avg', 'Ya_avg']
# engineered prediction inputs 
engineered_features = ['Wx', 'Wy', 'Day sin', 'Day cos', 'Year sin', 'Year cos']

### TRAIN/VAL/TEST SPLIT
train_years = [2013, 2014, 2015, 2016]
validation_years = [2017]
test_years = [2017]

### TRAINING PARAMETERS:
BATCH_SIZE = 512
N_EPOCHS = 100
N_NEURONS = 64

### FEATURE ENGINEERING PARAMETERS
MA_CONSTANT = 3 # moving average smoothing parameter
N_OUT = 3
N_PAST = 48

In [4]:
# define datasets
weather_link = '../weather_data/48.449583,5.586111.csv'
data_link = 'turbine_R80711_with_imputation.csv'

# read datasets
data, weather_data = processing.read_datasets(data_link, weather_link)

# merge with weather data
data = pd.merge(data, weather_data, on='Date_time')

FileNotFoundError: [Errno 2] No such file or directory: '../weather_data/48.449583,5.586111.csv'